In [13]:
dockerfile = """
FROM mcr.microsoft.com/azureml/aifx/stable-ubuntu2004-cu115-py38-torch1110:20220930.1

# Install pip dependencies
RUN pip install 'ipykernel~=6.0' \
                'azureml-core==1.46.0' \
				'azureml-dataset-runtime==1.46.0' \
                'azureml-defaults==1.46.0' \
				'azure-ml==0.0.1' \
				'azure-ml-component==0.9.13.post1' \
                'azureml-mlflow==1.46.0' \
		'azureml-contrib-services==1.46.0' \
                'azureml-telemetry==1.46.0' \
                'torch-tb-profiler~=0.4.0' \
				'py-spy==0.3.12'

RUN pip install \
        azure-ai-ml==0.1.0b5 \
        azureml-inference-server-http~=0.7.0 \
        inference-schema~=1.4.2.1 \
        MarkupSafe==2.0.1 \
	    regex \
	    pybind11 \
        'dalle-mini==0.1.1' \
        'flax==0.5.0' \
        'git+https://github.com/patil-suraj/vqgan-jax.git'

# Install jax[cuda]
RUN pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# Inference requirements
COPY --from=mcr.microsoft.com/azureml/o16n-base/python-assets:20220607.v1 /artifacts /var/
RUN /var/requirements/install_system_requirements.sh && \
    cp /var/configuration/rsyslog.conf /etc/rsyslog.conf && \
    cp /var/configuration/nginx.conf /etc/nginx/sites-available/app && \
    ln -sf /etc/nginx/sites-available/app /etc/nginx/sites-enabled/app && \
    rm -f /etc/nginx/sites-enabled/default
ENV SVDIR=/var/runit
ENV WORKER_TIMEOUT=400
EXPOSE 5001 8883 8888


"""

In [14]:
from azureml.core import Workspace, Environment
workspace = Workspace.from_config()

dalle_env = Environment(name = 'dall-e')
dalle_env.docker.enabled = True
dalle_env.docker.base_image = None
dalle_env.python.user_managed_dependencies = True
dalle_env.docker.base_dockerfile = dockerfile
dalle_env.inferencing_stack_version = "latest"
dalle_env.register(workspace=workspace)

# dalle_env.build_local(workspace, useDocker=True, pushImageToWorkspaceAcr=True)
dalle_env.build(workspace)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
Building a non-registered environment is not supported. Registering environment.
